In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('bonferroni_holm_ab_data.csv')

In [3]:
df

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18
...,...,...,...,...,...,...,...,...,...,...,...
618014,104 - 118,DAL - OKC,400874368.0,4.0,Randy Foye makes free throw 1 of 2,Randy Foye,playoffs,103 - 116,2015 - 2016,1,0:53
618015,104 - 118,DAL - OKC,400874368.0,4.0,Randy Foye makes free throw 2 of 2,Randy Foye,playoffs,103 - 117,2015 - 2016,1,0:53
618016,104 - 118,DAL - OKC,400874368.0,4.0,Kevin Durant makes technical free throw,Kevin Durant,playoffs,103 - 118,2015 - 2016,1,0:27
618017,104 - 118,DAL - OKC,400874368.0,4.0,Dwight Powell misses free throw 1 of 2,Dwight Powell,playoffs,103 - 118,2015 - 2016,0,0:26


Напишем функцию которая создаст отдельный датафрейм в котором будем записывать для каждого игрока его среднюю результативность (среднюю долю) попаданий во время регулярного чемпионата и playoff, а так же p-value

In [4]:
from statsmodels.stats.weightstats import ztest

In [5]:
# Создаем пустой датафрейм:
new_df = {
    'player': [],
    'regular_mean': [],
    'playoff_mean': [],
    'p_value': []
}

In [6]:
# Создаем цикл:
for player, group in df.groupby('player'):
    regular_shots = group[group['playoffs'] == 'regular']['shot_made'].values
    playoff_shots = group[group['playoffs'] == 'playoffs']['shot_made'].values

    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue

    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df['player'].append(player)
    new_df['regular_mean'].append(np.mean(regular_shots))
    new_df['playoff_mean'].append(np.mean(playoff_shots))
    new_df['p_value'].append(p_value)

переводим получившийся результат в датафрейм

In [7]:
new_df = pd.DataFrame(new_df)

посмотрим на 10 случайныйх игроков в новом датафрейме

In [8]:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
97,Ian Mahinmi,0.597627,0.607595,0.862398
81,Enes Kanter,0.763476,0.826087,0.327711
106,James Jones,0.843091,0.888889,0.289811
33,Brandon Roy,0.800587,0.808824,0.867596
17,Anthony Davis,0.779705,0.888889,0.116581
210,Rashard Lewis,0.821342,0.788043,0.275866
157,Leandro Barbosa,0.830808,0.728000,0.004251
52,D.J. Augustin,0.867177,0.839286,0.547825
79,Ed Davis,0.562997,0.578947,0.846240
201,Paul George,0.837629,0.797753,0.070627


посмотрим на кол-во игроков с p-value меньше 5%

In [9]:
new_df[new_df['p_value'] < 0.05].shape[0]

22

Расчеты готовы, но нам необходимо сравнить их с поправкой Бонферрони и методом Холма

In [10]:
# Импортируем функцию для множественной проверки гипотез:
from statsmodels.stats.multitest import multipletests

In [11]:
# Сравниваем результаты:
print(f'Bonferroni: {multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()}')
print(f'Holm: {multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()}')

Bonferroni: 1
Holm: 1


Видим что данной задаче 2 метода выдали один результат о колличестве игроков для которых стоит отвергнуть нулевую гипотизу. То есть тех у кого с 95% вероятностью отличается результативность во время плей-офф или регулярного чемпионата.

Одна мы видим, на сколько данное число игроков отличается от того, что мы получили без поправок на множественную проверку гипотез.

Так нам удалость кратно уменьшить ошибку первого рода